In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn 
from sklearn import preprocessing 
from tpot import TPOTRegressor   
from sklearn.datasets import load_digits  
from sklearn.cross_validation import train_test_split  

C:\Users\chethan\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\chethan\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory c:\users\chethan\anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: Deprecatio

In [2]:
import pandas as pd
import numpy as np
import urllib
from urllib import request
import sklearn 
from statistics import mean 
import statistics
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from datetime import time
from sklearn import feature_selection

In [3]:
df_train = pd.read_table("C:\\Users\\chethan\\Downloads\\historical_data1_Q12005.txt",sep='\|',names = ('fico', 'dt_first_pi', 'flag_fthb', 'dt_matr', 'cd_msa', 'mi_pct',
       'cnt_units', 'occpy_sts', 'cltv', 'dti', 'orig_upb', 'ltv',
       'int_rt', 'channel', 'ppmt_pnlty', 'prod_type', 'st', 'prop_type',
       'zipcode', 'id_loan', 'loan_purpose', 'orig_loan_term', 'cnt_borr',
       'seller_name', 'servicer_name', 'flag_sc', 'Year'),engine='python')

In [4]:
df_test = pd.read_table("C:\\Users\\chethan\\Downloads\\historical_data1_Q22005.txt",sep='\|',names = ('fico', 'dt_first_pi', 'flag_fthb', 'dt_matr', 'cd_msa', 'mi_pct',
       'cnt_units', 'occpy_sts', 'cltv', 'dti', 'orig_upb', 'ltv',
       'int_rt', 'channel', 'ppmt_pnlty', 'prod_type', 'st', 'prop_type',
       'zipcode', 'id_loan', 'loan_purpose', 'orig_loan_term', 'cnt_borr',
       'seller_name', 'servicer_name', 'flag_sc', 'Year'),engine='python')

In [5]:
#######Data Cleaning##########
def cleandata(df):


    
    #To convert the categorical to numerical
    
    df_sccstats = pd.get_dummies(df['occpy_sts'])
    df_sccstats.columns = ['OS_I', 'OS_P', 'OS_S']
    del df_sccstats['OS_I']
    print(df_sccstats.head())
    
    df_channel = pd.get_dummies(df['channel'])
    df_channel.columns = ['C_B', 'C_C', 'C_R','C_T']
    del df_channel['C_B']
    print(df_channel.head())
    
    df_ppmt = pd.get_dummies(df['ppmt_pnlty'])
    df_ppmt.columns = ['ppmt_N0', 'ppmt_YES']
    del df_ppmt['ppmt_YES']
    print(df_ppmt.head())
    
    df_proptype = pd.get_dummies(df['prop_type'])
    del df_proptype['99']
    print(df_proptype.head())
    
    df_loanpurpose = pd.get_dummies(df['loan_purpose'])
    print(df_loanpurpose.head())
    
    df_prodtype = pd.get_dummies(df['prod_type'])
    print(df_prodtype.head())
    
    df_fthb = pd.get_dummies(df['flag_fthb'])
    del df_fthb['9']
    df_fthb.columns = ['fthb_N', 'fthb_Y']
    print(df_fthb.head())
    
    df = df.join(df_sccstats)
    df = df.join(df_channel)
    df = df.join(df_ppmt)
    df = df.join(df_prodtype)
    df = df.join(df_proptype)
    df = df.join(df_loanpurpose)
    df = df.join(df_fthb)
    del df['occpy_sts']
    del df['channel']
    del df['ppmt_pnlty']
    del df['prod_type']
    del df['prop_type']
    del df['loan_purpose']
    del df['flag_fthb']
    
    #To fill the missing values
    df['cd_msa'].fillna(value=0,inplace= True)
    df['mi_pct'].fillna(value=999,inplace= True)
    df['cnt_units'].fillna(value=99,inplace= True)
    df['cltv'].fillna(value=999,inplace= True)
    df['dti'].fillna(value=999,inplace= True)
    df['ltv'].fillna(value=999,inplace= True)
    df.fillna(value=0,inplace= True)

    #dropping these features
    del df['id_loan']
    del df['seller_name']
    del df['servicer_name']
    del df['st']
    print("Final columns : ")
    print(df.columns)
    return df


In [6]:
#Cleaned Final DataFrame
df_train_cleaned = cleandata(df_train)
df_test_cleaned = cleandata(df_test)

   OS_P  OS_S
0     1     0
1     1     0
2     1     0
3     0     1
4     1     0
   C_C  C_R  C_T
0    0    1    0
1    0    1    0
2    0    1    0
3    0    1    0
4    0    1    0
   ppmt_N0
0        1
1        1
2        1
3        1
4        1
   CO  CP  MH  PU  SF
0   0   0   0   0   1
1   0   0   0   0   1
2   0   0   0   0   1
3   1   0   0   0   0
4   0   0   0   0   1
   C  N  P
0  1  0  0
1  0  1  0
2  0  0  1
3  0  0  1
4  1  0  0
   FRM
0    1
1    1
2    1
3    1
4    1
   fthb_N  fthb_Y
0       1       0
1       1       0
2       1       0
3       1       0
4       1       0
Final columns : 
Index(['fico', 'dt_first_pi', 'dt_matr', 'cd_msa', 'mi_pct', 'cnt_units',
       'cltv', 'dti', 'orig_upb', 'ltv', 'int_rt', 'zipcode', 'orig_loan_term',
       'cnt_borr', 'flag_sc', 'Year', 'OS_P', 'OS_S', 'C_C', 'C_R', 'C_T',
       'ppmt_N0', 'FRM', 'CO', 'CP', 'MH', 'PU', 'SF', 'C', 'N', 'P', 'fthb_N',
       'fthb_Y'],
      dtype='object')
   OS_P  OS_S
0     1     0
1     

In [7]:
yt = df_train_cleaned['int_rt']
del df_train_cleaned['int_rt']
ytest = df_test_cleaned['int_rt']
del df_test_cleaned['int_rt']
X_train = df_train_cleaned
X_test = df_test_cleaned
y_train = yt
y_test = ytest

In [9]:
tpot = TPOTRegressor(generations=3, verbosity=2)  
tpot.fit(X_train, y_train)  
 

C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: -0.07463868521317207
Generation 2 - Current best internal CV score: -0.07463868521317207
Generation 3 - Current best internal CV score: -0.07463868521317207

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.3, min_samples_leaf=6, min_samples_split=4, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=3,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=100,
       random_state=None, scoring=None, subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)

In [11]:
print(tpot.score(X_test, y_test))

-0.08672433863806606


In [16]:
ypredtpot_test = tpot.predict(X_test)

In [17]:
ypredtpot_train = tpot.predict(X_train)

In [14]:
def mae(org,pred):
    return mean_absolute_error(org,pred)
def rmse(org,pred):
    return np.sqrt(mean_squared_error(org,pred))
def mape(org,pred):
    y_true, y_pred = np.array(org), np.array(pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
def scores(model,yorg,ypred):
    print("The scores for the model : ",model)
    print("MAE    :    ",mae(yorg,ypred))
    print("RMSE   :    ",rmse(yorg,ypred))
    print("MAPE   :    ",mape(yorg,ypred))

In [18]:
print("Scores for test")
scores("TPOT",y_test,ypredtpot_test)

Scores for test
The scores for the model :  TPOT
MAE    :     0.22545943023682835
RMSE   :     0.294489963560842
MAPE   :     3.9022006060336705


In [19]:
print("Scores for train")
scores("TPOT",y_train,ypredtpot_train)

Scores for train
The scores for the model :  TPOT
MAE    :     0.15370824295682756
RMSE   :     0.21097243449412878
MAPE   :     2.718942174159185


In [20]:
tpot.get_params

<bound method BaseEstimator.get_params of TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=3,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=100,
       random_state=None, scoring=None, subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)>

In [21]:
tpot.fitted_pipeline_

Pipeline(memory=None,
     steps=[('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.3, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=6, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [22]:
tpot.export('tpot_exported_pipeline.py')

True